In [117]:
import urllib
import xmltodict
import json
import pandas as pd
from datetime import datetime

In [118]:
!pip install  openpyxl

In [119]:
import  openpyxl


In [120]:
today = datetime.today().strftime("%Y%m%d")

In [121]:
service_key = 'du7sxfdZ1IgPVwgcIN89XSOUVEow0Eho%2Bv%2B%2BapFpI7P0eHcZazptxafwktGn2BaT3zWsvCy8TE5MNS9suXGZ2g%3D%3D'
base_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson'
pageNo = '1'
numOfRows = '500'
startCreateDt = "20200101"
endCreateDt = today
request_url = base_url + '?' + "serviceKey=" + service_key+ '&pageNo=' + pageNo + '&numOfRows=' + numOfRows + '&startCreateDt=' + startCreateDt + '&endCreateDt=' + endCreateDt 
response = urllib.request.urlopen(request_url) 
json_str = response.read().decode("utf-8") 
response.status



200

In [122]:
data_dict=xmltodict.parse(json_str) #dict로 만들기

In [123]:
json_data = json.dumps(data_dict,ensure_ascii = False) #json으로 만들기

In [124]:
corona_json=json.loads(json_data) #json데이터 로드

In [153]:
corona_json

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'},
  'body': {'items': {'item': [{'confCase': '568',
      'confCaseRate': '2.41',
      'createDt': '2020-09-27 14:22:48.573',
      'criticalRate': '0',
      'death': '0',
      'deathRate': '0.00',
      'gubun': '0-9',
      'seq': '3632',
      'updateDt': 'null'},
     {'confCase': '1295',
      'confCaseRate': '5.48',
      'createDt': '2020-09-27 14:22:48.573',
      'criticalRate': '0',
      'death': '0',
      'deathRate': '0.00',
      'gubun': '10-19',
      'seq': '3631',
      'updateDt': 'null'},
     {'confCase': '4741',
      'confCaseRate': '20.08',
      'createDt': '2020-09-27 14:22:48.572',
      'criticalRate': '0',
      'death': '0',
      'deathRate': '0.00',
      'gubun': '20-29',
      'seq': '3630',
      'updateDt': 'null'},
     {'confCase': '2878',
      'confCaseRate': '12.19',
      'createDt': '2020-09-27 14:22:48.572',
      'criticalRate': '0.07',
      'death': '2',
      

In [158]:
# 데이터프레임으로만들기

result = pd.DataFrame()

for item in corona_json['response']['body']["items"]['item']:
    s = pd.Series([item["createDt"],item["gubun"], int(item["death"]), float(item["deathRate"]) ,int(item["confCase"]), float(item["confCaseRate"])] 
                  ,index = ["등록일","구분","사망자","사망률", "확진자", "확진률"])
    result = result.append(s, ignore_index = True)   


In [159]:
result.head(5)

,구분,등록일,사망률,사망자,확진률,확진자
0,0-9,2020-09-27 14:22:48.573,0.0,0.0,2.41,568.0
1,10-19,2020-09-27 14:22:48.573,0.0,0.0,5.48,1295.0
2,20-29,2020-09-27 14:22:48.572,0.0,0.0,20.08,4741.0
3,30-39,2020-09-27 14:22:48.572,0.5,2.0,12.19,2878.0
4,40-49,2020-09-27 14:22:48.572,1.0,4.0,13.32,3146.0


In [160]:
#일일 확진자 계산

daily_ct_gu_list =[]
for i in range(len(corona_json['response']['body']["items"]["item"])):
    temp = corona_json['response']['body']["items"]["item"]
    try :
        daily_ct_gu = (int(temp[i]["confCase"])- int(temp[i+11]["confCase"]))
        daily_ct_gu_list.append(int(daily_ct_gu))
    except :
        daily_ct_gu_list.append(0)
result["일일확진자"] = daily_ct_gu_list

In [161]:
result.head(22)

,구분,등록일,사망률,사망자,확진률,확진자,일일확진자
0,0-9,2020-09-27 14:22:48.573,0.00,0.0,2.41,568.0,1
1,10-19,2020-09-27 14:22:48.573,0.00,0.0,5.48,1295.0,4
2,20-29,2020-09-27 14:22:48.572,0.00,0.0,20.08,4741.0,14
3,30-39,2020-09-27 14:22:48.572,0.50,2.0,12.19,2878.0,16
4,40-49,2020-09-27 14:22:48.572,1.00,4.0,13.32,3146.0,12
5,50-59,2020-09-27 14:22:48.572,4.49,18.0,18.55,4380.0,19
6,60-69,2020-09-27 14:22:48.572,10.97,44.0,15.87,3748.0,20
7,70-79,2020-09-27 14:22:48.572,32.42,130.0,8.02,1894.0,7
8,80 이상,2020-09-27 14:22:48.572,50.62,203.0,4.07,961.0,2
9,여성,2020-09-27 14:22:48.572,46.88,188.0,54.11,12777.0,53


In [162]:
#일일사망자 계산
daily_ct_gu_list =[]
for i in range(len(corona_json['response']['body']["items"]["item"])):
    temp = corona_json['response']['body']["items"]["item"]
    try :
        daily_ct_gu = (int(temp[i]["death"])- int(temp[i+11]["death"]))
        daily_ct_gu_list.append(int(daily_ct_gu))
    except :
        daily_ct_gu_list.append(0)
result["일일사망자"] = daily_ct_gu_list

In [163]:
result.head(22)

,구분,등록일,사망률,사망자,확진률,확진자,일일확진자,일일사망자
0,0-9,2020-09-27 14:22:48.573,0.00,0.0,2.41,568.0,1,0
1,10-19,2020-09-27 14:22:48.573,0.00,0.0,5.48,1295.0,4,0
2,20-29,2020-09-27 14:22:48.572,0.00,0.0,20.08,4741.0,14,0
3,30-39,2020-09-27 14:22:48.572,0.50,2.0,12.19,2878.0,16,0
4,40-49,2020-09-27 14:22:48.572,1.00,4.0,13.32,3146.0,12,0
5,50-59,2020-09-27 14:22:48.572,4.49,18.0,18.55,4380.0,19,0
6,60-69,2020-09-27 14:22:48.572,10.97,44.0,15.87,3748.0,20,1
7,70-79,2020-09-27 14:22:48.572,32.42,130.0,8.02,1894.0,7,0
8,80 이상,2020-09-27 14:22:48.572,50.62,203.0,4.07,961.0,2,1
9,여성,2020-09-27 14:22:48.572,46.88,188.0,54.11,12777.0,53,0


In [164]:
# 나이 기준 데이터 분리
del_list = result[result["구분"].isin(["남성","여성"])].index  ##성별데이터지우기
year_result = result
for d in del_list:
    year_result = year_result.drop(d, inplace = False )

In [165]:
year_result.head(22)

,구분,등록일,사망률,사망자,확진률,확진자,일일확진자,일일사망자
0,0-9,2020-09-27 14:22:48.573,0.00,0.0,2.41,568.0,1,0
1,10-19,2020-09-27 14:22:48.573,0.00,0.0,5.48,1295.0,4,0
2,20-29,2020-09-27 14:22:48.572,0.00,0.0,20.08,4741.0,14,0
3,30-39,2020-09-27 14:22:48.572,0.50,2.0,12.19,2878.0,16,0
4,40-49,2020-09-27 14:22:48.572,1.00,4.0,13.32,3146.0,12,0
5,50-59,2020-09-27 14:22:48.572,4.49,18.0,18.55,4380.0,19,0
6,60-69,2020-09-27 14:22:48.572,10.97,44.0,15.87,3748.0,20,1
7,70-79,2020-09-27 14:22:48.572,32.42,130.0,8.02,1894.0,7,0
8,80 이상,2020-09-27 14:22:48.572,50.62,203.0,4.07,961.0,2,1
11,0-9,2020-09-26 14:09:05.525,0.00,0.0,2.41,567.0,1,0


In [166]:
year_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1549 entries, 0 to 1890
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   구분      1549 non-null   object 
 1   등록일     1549 non-null   object 
 2   사망률     1549 non-null   float64
 3   사망자     1549 non-null   float64
 4   확진률     1549 non-null   float64
 5   확진자     1549 non-null   float64
 6   일일확진자   1549 non-null   int64  
 7   일일사망자   1549 non-null   int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 108.9+ KB


In [167]:
#등록일 데이터타입바꾸고 인덱스로 바꾸기
year_result["등록일"] = pd.to_datetime(year_result["등록일"])
year_result = year_result.set_index("등록일", inplace = False)

In [168]:
year_result

,구분,사망률,사망자,확진률,확진자,일일확진자,일일사망자
등록일,,,,,,,
2020-09-27 14:22:48.573,0-9,0.00,0.0,2.41,568.0,1,0
2020-09-27 14:22:48.573,10-19,0.00,0.0,5.48,1295.0,4,0
2020-09-27 14:22:48.572,20-29,0.00,0.0,20.08,4741.0,14,0
2020-09-27 14:22:48.572,30-39,0.50,2.0,12.19,2878.0,16,0
2020-09-27 14:22:48.572,40-49,1.00,4.0,13.32,3146.0,12,0
...,...,...,...,...,...,...,...
2020-04-02 19:02:48.480,40-49,0.62,1.0,13.41,1312.0,0,0
2020-04-02 19:02:48.480,50-59,6.17,10.0,18.91,1851.0,0,0
2020-04-02 19:02:48.480,60-69,13.58,22.0,12.62,1235.0,0,0


In [169]:
#나이 기준 데이터 저장하기
year_result.to_excel('corona_year_result.xlsx',index =False, encoding= "euc-kr")

In [170]:
#나이 기준 데이터 월별 평균 구하기
m_year_result = year_result.resample("M").mean()

In [171]:
m_year_result

,사망률,사망자,확진률,확진자,일일확진자,일일사망자
등록일,,,,,,
2020-04-30,11.066150,24.929204,11.066283,1170.548673,-11.915929,0.340708
2020-05-31,11.111333,28.951852,11.111185,1226.251852,2.603704,0.085185
2020-06-30,11.111230,30.861111,11.111468,1352.293651,5.285714,0.047619
2020-07-31,11.111219,32.394265,11.111290,1511.774194,5.394265,0.068100
2020-08-31,11.111434,33.810036,11.110932,1790.691756,20.222222,0.082437
2020-09-30,11.111193,40.144033,11.110864,2461.267490,15.078189,0.316872


In [172]:
#나이 기준 데이터 월별 평균 데이터 저장하기
m_year_result.to_excel('m_corona_year_result.xlsx',index =False, encoding= "euc-kr")

In [173]:
#성별 기준 데이터 분리하기
se_list = result[result["구분"].isin(["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80 이상"])].index
gen_result = result
for num in se_list:
    gen_result.drop(num, inplace = False)


In [174]:
gen_result.head(5)

,구분,등록일,사망률,사망자,확진률,확진자,일일확진자,일일사망자
0,0-9,2020-09-27 14:22:48.573,0.0,0.0,2.41,568.0,1,0
1,10-19,2020-09-27 14:22:48.573,0.0,0.0,5.48,1295.0,4,0
2,20-29,2020-09-27 14:22:48.572,0.0,0.0,20.08,4741.0,14,0
3,30-39,2020-09-27 14:22:48.572,0.5,2.0,12.19,2878.0,16,0
4,40-49,2020-09-27 14:22:48.572,1.0,4.0,13.32,3146.0,12,0


In [175]:
#성별 기준 데이터 저장하기
gen_result.to_excel('gender_corona_result.xlsx',index =False, encoding= "euc-kr")

In [176]:
#등록일 데이터타입바꾸기
gen_result["등록일"] = pd.to_datetime(gen_result["등록일"])
gen_result = gen_result.set_index("등록일", inplace = False)
gen_result

,구분,사망률,사망자,확진률,확진자,일일확진자,일일사망자
등록일,,,,,,,
2020-09-27 14:22:48.573,0-9,0.00,0.0,2.41,568.0,1,0
2020-09-27 14:22:48.573,10-19,0.00,0.0,5.48,1295.0,4,0
2020-09-27 14:22:48.572,20-29,0.00,0.0,20.08,4741.0,14,0
2020-09-27 14:22:48.572,30-39,0.50,2.0,12.19,2878.0,16,0
2020-09-27 14:22:48.572,40-49,1.00,4.0,13.32,3146.0,12,0
...,...,...,...,...,...,...,...
2020-04-02 19:02:48.480,60-69,13.58,22.0,12.62,1235.0,0,0
2020-04-02 19:02:48.480,70-79,28.40,46.0,6.65,651.0,0,0
2020-04-02 19:02:48.480,80 이상,50.62,82.0,4.52,442.0,0,0


In [177]:
m_gen_result = gen_result.resample("M").mean()

In [178]:
m_gen_result

,사망률,사망자,확진률,확진자,일일확진자,일일사망자
등록일,,,,,,
2020-04-30,18.119384,40.822464,18.119493,1916.981884,7.094203,0.615942
2020-05-31,18.182000,47.375758,18.181879,2006.593939,4.260606,0.139394
2020-06-30,18.181916,50.564935,18.182110,2212.844156,8.649351,0.077922
2020-07-31,18.181965,53.008798,18.181965,2473.812317,8.826979,0.111437
2020-08-31,18.182082,55.325513,18.181672,2930.222874,33.090909,0.134897
2020-09-30,18.181886,65.690236,18.181616,4027.528620,24.673401,0.518519


In [179]:
#성별 기준 월별 평균 데이터 저장하기
m_gen_result.to_excel("m_corona_gender_result.xlsx", index =False, encoding= "euc-kr")